In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("../../")

#spectrome modules
# from spectrome.utils import functions, path
# from spectrome.brain import Brain
# from spectrome.forward import ntf_local_stimulus as nt_stim
# from spectrome.forward import ntf_local as nt_nostim
# from spectrome.forward import ntf_local_stimulus as nt_stim

from scipy.io import loadmat
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import fdrcorrection

from scipy.io import savemat
import pickle as pkl

In [2]:
data1=loadmat("../../../../BRACCIO_Database_PWelch_AvgTrials_MI_RestingState_Strat2.mat")

In [3]:
fvec = np.squeeze(data1['Freqs_MI_Strat2_DK'][0])[3:30]
df=pd.read_excel('../../../../spectrome-bci/spectrome/ListROIs_DK.xlsx')

In [4]:
all_regions=[]
all_regions_idx=[]
for i,r in df.iterrows():
    all_regions_idx.append(i)
    all_regions.append(r['ROI'].strip("'"))

In [5]:
##strategy 1--modeled data from subject 11 to 25
x_mi_all={}
x_rest_all={}

num=1
x_mi_all[num]=np.zeros((68, 10, 26))
x_rest_all[num]=np.zeros((68, 10, 26))

for i in range(26):
    mi_fname = f'../../bci_results_apd/bci_results/validation_strat1/mi_allsubs_f/mi_nostim_mse_db_fvec330_{i}.p'
    with open(mi_fname, 'rb') as f:
        x_mi_all[num][:,:,i]=np.array(pkl.load(f))
    
    rest_fname = f'../../bci_results_apd/bci_results/validation_strat1/rest_allsubs_f0/rest_nostim_mse_db_fvec330_{i}.p'
    with open(rest_fname, 'rb') as f:
        x_rest_all[num][:,:,i]=np.array(pkl.load(f))

In [6]:
x_mi_all[1][:,2, 11:]

array([[30.        , 10.82599006,  5.29788976, ...,  5.05243254,
         6.13559524, 17.35655339],
       [ 7.68191251,  9.81294208, 24.8593523 , ...,  7.54985686,
         5.        , 13.69477351],
       [29.83771748, 29.06295955, 13.06368152, ..., 29.91565528,
         6.22239945,  8.70327256],
       ...,
       [30.        , 18.34644355, 24.45009783, ...,  5.        ,
        16.48317128, 13.4124424 ],
       [ 5.03910452, 30.        , 12.63978999, ..., 12.93189544,
         6.0455286 , 13.30408478],
       [ 5.93603932,  5.        ,  7.10447268, ..., 10.37632828,
        10.1969797 ,  6.91827474]])

In [7]:
##Batch-2
#to store p-values--(gei, gii, tau_e, tau_i)-across all subjects/performers
### for MI V. Rest
##for each region across subjects
sess=1
pval_all_reg_top = np.zeros((len(all_regions_idx), 4))
for i in range(len(all_regions_idx)):
    pval_all_reg_top[i,0]=wilcoxon(x_mi_all[sess][all_regions_idx[i],0, 11:],x_rest_all[sess][all_regions_idx[i],0,11:])[1] #gei
    pval_all_reg_top[i,1]=wilcoxon(x_mi_all[sess][all_regions_idx[i],1, 11:],x_rest_all[sess][all_regions_idx[i],1,11:])[1] #gii
    pval_all_reg_top[i,2]=wilcoxon(x_mi_all[sess][all_regions_idx[i],2,11:],x_rest_all[sess][all_regions_idx[i],2,11:])[1] #tau_e
    pval_all_reg_top[i,3]=wilcoxon(x_mi_all[sess][all_regions_idx[i],3,11:],x_rest_all[sess][all_regions_idx[i],3,11:])[1] #tau_i

# FDR-corrected p-values 
pfdr=np.zeros((len(all_regions_idx), 4))
for i in range(4):
    pfdr[:,i]=fdrcorrection(pval_all_reg_top[:, i])[1]

significant_gei= np.where(pfdr[:,0]<0.05)[0]
significant_gii= np.where(pfdr[:,1]<0.05)[0]
significant_tau_e= np.where(pfdr[:,2]<0.05)[0]
significant_tau_i= np.where(pfdr[:,3]<0.05)[0]

print("Significant ROIs for gei (pfdr <0.01):", [all_regions[i] for i in significant_gei])
print("Significant ROIs for gii (pfdr < 0.05):", [all_regions[i] for i in significant_gii])
print("Significant ROIs for tau_e (pfdr < 0.05):", [all_regions[i] for i in significant_tau_e])
print("Significant ROIs for tau_i (pfdr < 0.05):", [all_regions[i]  for i in significant_tau_i])

print(significant_gii.shape,significant_tau_e.shape)

#FDR-corrected p-values matrix for inspection
# print("FDR Corrected p-values matrix for Top Performers:\n", pfdr)

Significant ROIs for gei (pfdr <0.01): []
Significant ROIs for gii (pfdr < 0.05): ['bankssts L', 'caudalanteriorcingulate L', 'caudalanteriorcingulate R', 'caudalmiddlefrontal L', 'caudalmiddlefrontal R', 'cuneus R', 'entorhinal L', 'frontalpole L', 'frontalpole R', 'insula L', 'medialorbitofrontal L', 'paracentral R', 'parahippocampal R', 'parsopercularis R', 'parstriangularis L', 'pericalcarine L', 'posteriorcingulate L', 'posteriorcingulate R', 'precuneus L', 'precuneus R', 'rostralanteriorcingulate L', 'rostralanteriorcingulate R', 'rostralmiddlefrontal L', 'superiorfrontal R', 'superiorparietal R', 'temporalpole L', 'temporalpole R']
Significant ROIs for tau_e (pfdr < 0.05): ['lateralorbitofrontal L', 'lateralorbitofrontal R', 'medialorbitofrontal L', 'medialorbitofrontal R', 'rostralanteriorcingulate R']
Significant ROIs for tau_i (pfdr < 0.05): []
(27,) (5,)


In [14]:
##Batch-1
#to store p-values--(gei, gii, tau_e, tau_i)-across all subjects/performers
### for MI V. Rest
##for each region across subjects
sess=1
pval_all_reg_top = np.zeros((len(all_regions_idx), 4))
for i in range(len(all_regions_idx)):
    pval_all_reg_top[i,0]=wilcoxon(x_mi_all[sess][all_regions_idx[i],0, :10],x_rest_all[sess][all_regions_idx[i],0,:10])[1] #gei
    pval_all_reg_top[i,1]=wilcoxon(x_mi_all[sess][all_regions_idx[i],1, :10],x_rest_all[sess][all_regions_idx[i],1,:10])[1] #gii
    pval_all_reg_top[i,2]=wilcoxon(x_mi_all[sess][all_regions_idx[i],2,:10],x_rest_all[sess][all_regions_idx[i],2,:10])[1] #tau_e
    pval_all_reg_top[i,3]=wilcoxon(x_mi_all[sess][all_regions_idx[i],3,:10],x_rest_all[sess][all_regions_idx[i],3,:10])[1] #tau_i

# FDR-corrected p-values 
pfdr=np.zeros((len(all_regions_idx), 4))
for i in range(4):
    pfdr[:,i]=fdrcorrection(pval_all_reg_top[:, i])[1]

significant_gei= np.where(pfdr[:,0]<0.05)[0]
significant_gii= np.where(pfdr[:,1]<0.05)[0]
significant_tau_e= np.where(pfdr[:,2]<0.05)[0]
significant_tau_i= np.where(pfdr[:,3]<0.05)[0]

print("Significant ROIs for gei (pfdr <0.01):", [all_regions[i] for i in significant_gei])
print("Significant ROIs for gii (pfdr < 0.05):", [all_regions[i] for i in significant_gii])
print("Significant ROIs for tau_e (pfdr < 0.05):", [all_regions[i] for i in significant_tau_e])
print("Significant ROIs for tau_i (pfdr < 0.05):", [all_regions[i]  for i in significant_tau_i])

print(significant_gii.shape,significant_tau_e.shape)

#FDR-corrected p-values matrix for inspection
# print("FDR Corrected p-values matrix for Top Performers:\n", pfdr)

Significant ROIs for gei (pfdr <0.01): []
Significant ROIs for gii (pfdr < 0.05): []
Significant ROIs for tau_e (pfdr < 0.05): []
Significant ROIs for tau_i (pfdr < 0.05): []
(0,) (0,)


In [15]:
##all
#to store p-values--(gei, gii, tau_e, tau_i)-across all subjects/performers
### for MI V. Rest
##for each region across subjects
sess=1
pval_all_reg_top = np.zeros((len(all_regions_idx), 4))
for i in range(len(all_regions_idx)):
    pval_all_reg_top[i,0]=wilcoxon(x_mi_all[sess][all_regions_idx[i],0, :],x_rest_all[sess][all_regions_idx[i],0,:])[1] #gei
    pval_all_reg_top[i,1]=wilcoxon(x_mi_all[sess][all_regions_idx[i],1, :],x_rest_all[sess][all_regions_idx[i],1,:])[1] #gii
    pval_all_reg_top[i,2]=wilcoxon(x_mi_all[sess][all_regions_idx[i],2,:],x_rest_all[sess][all_regions_idx[i],2,:])[1] #tau_e
    pval_all_reg_top[i,3]=wilcoxon(x_mi_all[sess][all_regions_idx[i],3,:],x_rest_all[sess][all_regions_idx[i],3,:])[1] #tau_i

# FDR-corrected p-values 
pfdr=np.zeros((len(all_regions_idx), 4))
for i in range(4):
    pfdr[:,i]=fdrcorrection(pval_all_reg_top[:, i])[1]

significant_gei= np.where(pfdr[:,0]<0.05)[0]
significant_gii= np.where(pfdr[:,1]<0.05)[0]
significant_tau_e= np.where(pfdr[:,2]<0.05)[0]
significant_tau_i= np.where(pfdr[:,3]<0.05)[0]

print("Significant ROIs for gei (pfdr <0.01):", [all_regions[i] for i in significant_gei])
print("Significant ROIs for gii (pfdr < 0.05):", [all_regions[i] for i in significant_gii])
print("Significant ROIs for tau_e (pfdr < 0.05):", [all_regions[i] for i in significant_tau_e])
print("Significant ROIs for tau_i (pfdr < 0.05):", [all_regions[i]  for i in significant_tau_i])

print(significant_gii.shape,significant_tau_e.shape)

#FDR-corrected p-values matrix for inspection
# print("FDR Corrected p-values matrix for Top Performers:\n", pfdr)

Significant ROIs for gei (pfdr <0.01): []
Significant ROIs for gii (pfdr < 0.05): ['bankssts L', 'caudalanteriorcingulate L', 'caudalanteriorcingulate R', 'caudalmiddlefrontal L', 'caudalmiddlefrontal R', 'cuneus L', 'cuneus R', 'entorhinal L', 'entorhinal R', 'frontalpole L', 'frontalpole R', 'fusiform R', 'inferiorparietal R', 'insula L', 'insula R', 'isthmuscingulate L', 'isthmuscingulate R', 'lateraloccipital R', 'lateralorbitofrontal L', 'lateralorbitofrontal R', 'lingual L', 'medialorbitofrontal L', 'medialorbitofrontal R', 'paracentral R', 'parahippocampal L', 'parahippocampal R', 'parsopercularis L', 'parsopercularis R', 'parsorbitalis L', 'parstriangularis L', 'parstriangularis R', 'pericalcarine L', 'postcentral R', 'posteriorcingulate L', 'posteriorcingulate R', 'precentral L', 'precuneus L', 'precuneus R', 'rostralanteriorcingulate L', 'rostralanteriorcingulate R', 'rostralmiddlefrontal L', 'rostralmiddlefrontal R', 'superiorfrontal L', 'superiorfrontal R', 'superiorparieta